# Basic Training Loop

In [1]:
import os
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

### 准备数据

In [2]:

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

### 定义模型

In [3]:

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

/opt/homebrew/anaconda3/envs/colab/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 定义损失函数和优化器

In [4]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

### 实现自定义Training Loops

In [5]:
epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')

Start of epoch 1
Epoch 1 Step 0: Loss = 2.303683280944824
Epoch 1 Step 200: Loss = 0.36938774585723877
Epoch 1 Step 400: Loss = 0.18165946006774902
Epoch 1 Step 600: Loss = 0.1624484360218048
Epoch 1 Step 800: Loss = 0.1730903685092926
Epoch 1 Step 1000: Loss = 0.41516339778900146
Epoch 1 Step 1200: Loss = 0.189113587141037
Epoch 1 Step 1400: Loss = 0.2236395925283432
Epoch 1 Step 1600: Loss = 0.2254944145679474
Epoch 1 Step 1800: Loss = 0.14914561808109283
Start of epoch 2
Epoch 2 Step 0: Loss = 0.08540712296962738
Epoch 2 Step 200: Loss = 0.16500020027160645
Epoch 2 Step 400: Loss = 0.08249332010746002
Epoch 2 Step 600: Loss = 0.03424878418445587
Epoch 2 Step 800: Loss = 0.0693032518029213
Epoch 2 Step 1000: Loss = 0.24864672124385834
Epoch 2 Step 1200: Loss = 0.12689433991909027
Epoch 2 Step 1400: Loss = 0.13711830973625183
Epoch 2 Step 1600: Loss = 0.2093067467212677
Epoch 2 Step 1800: Loss = 0.10138417780399323


# 添加 Accuracy 测量

### 设置环境

In [6]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

### 定义模型

In [7]:

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


### 定义损失函数，优化器和Metric

In [8]:

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


### 自定义Training Loop

In [9]:
epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.390378952026367 Accuracy = 0.0625
Epoch 1 Step 200: Loss = 0.41103625297546387 Accuracy = 0.8327114582061768
Epoch 1 Step 400: Loss = 0.18913236260414124 Accuracy = 0.8666614890098572
Epoch 1 Step 600: Loss = 0.19121761620044708 Accuracy = 0.8824355006217957
Epoch 1 Step 800: Loss = 0.1919010579586029 Accuracy = 0.8948969841003418
Epoch 1 Step 1000: Loss = 0.41956281661987305 Accuracy = 0.9018169045448303
Epoch 1 Step 1200: Loss = 0.2165583223104477 Accuracy = 0.908331573009491
Epoch 1 Step 1400: Loss = 0.2494545578956604 Accuracy = 0.9131200909614563
Epoch 1 Step 1600: Loss = 0.19828134775161743 Accuracy = 0.9164584875106812
Epoch 1 Step 1800: Loss = 0.1927342712879181 Accuracy = 0.9203567504882812
Start of epoch 2
Epoch 2 Step 0: Loss = 0.06752408295869827 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.17725607752799988 Accuracy = 0.9597325921058655
Epoch 2 Step 400: Loss = 0.10160834342241287 Accuracy = 0.9573721885681152
Epoch 2 Step 600: Loss =

# Custom Callback for Advanced Logging

In [14]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


In [15]:
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [16]:
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [19]:

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_sta

Start of epoch 1
Epoch 1 Step 0: Loss = 2.327249526977539 Accuracy = 0.15625
Epoch 1 Step 200: Loss = 0.36825716495513916 Accuracy = 0.8404850959777832
Epoch 1 Step 400: Loss = 0.16387075185775757 Accuracy = 0.8716490268707275
Epoch 1 Step 600: Loss = 0.14201074838638306 Accuracy = 0.885503351688385
Epoch 1 Step 800: Loss = 0.18003690242767334 Accuracy = 0.896847665309906
Epoch 1 Step 1000: Loss = 0.36329495906829834 Accuracy = 0.9035027623176575
Epoch 1 Step 1200: Loss = 0.17180293798446655 Accuracy = 0.9097887277603149
Epoch 1 Step 1400: Loss = 0.2600264251232147 Accuracy = 0.9145476222038269
Epoch 1 Step 1600: Loss = 0.2697421610355377 Accuracy = 0.9177076816558838
Epoch 1 Step 1800: Loss = 0.18687570095062256 Accuracy = 0.9215887188911438
End of epoch 1, loss: 0.035480670630931854, accuracy: 0.9235166907310486
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07216691970825195 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.1736968457698822 Accuracy = 0.9597325921058655
Epoch 2 Step 400: Loss = 

# 添加隐藏层

In [20]:
from tensorflow.keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In [21]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In [22]:
model = Model(inputs=input_layer, outputs=output_layer)

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - accuracy: 0.4840 - loss: 0.7009 
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.5343 - loss: 0.6909
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - accuracy: 0.5175 - loss: 0.6935
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.5467 - loss: 0.6915
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - accuracy: 0.5273 - loss: 0.6884
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.5647 - loss: 0.6842
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.5291 - loss: 0.6872
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.5707 - loss: 0.6813
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.5537 - loss: 0.6849
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.5828 - loss: 0.6794


In [25]:
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4735 - loss: 0.6996 
Test loss: 0.6979187726974487
Test accuracy: 0.5099999904632568
